In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *

In [ ]:
w_central = central_wavelength()

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_7.csv', sep=',', header=1,
    converters={0: int, 1: int, 2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop bad rows due to bad query
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
w_lya = 1215.67 # A

N_sources = len(cat['FLUX_AUTO'])
z_nb_Arr = w_central[:-4] / w_lya - 1
print(N_sources)

In [ ]:
pm_mask = mask_proper_motion(cat)

In [ ]:
# Lya search
cont_est_mat, cont_err_mat = estimate_continuum(pm_flx, pm_err)
line = is_there_line(pm_flx, pm_err, cont_est_mat, cont_err_mat, 30, mask=pm_mask)
lya_lines = np.array(identify_lines(line, pm_flx, first=True))

In [ ]:
cont_est_mat

In [ ]:
len(np.where(np.array(lya_lines) > 0)[0])

In [ ]:
for i, src in enumerate(np.where(lya_lines > 0)[0]):
    if i == 21: break
    fig, ax = plt.subplots(figsize=(10, 4))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])
    ax.scatter(w_central[lya_lines[src]], cont_est_mat[lya_lines[src], src])
    print(cont_est_mat[lya_lines[src], src])
    plt.show()